Comprendiamo insieme cosa possono contenere i file NetCDF e come possono essere gestiti con Python

## Prima di iniziare


### Librerie
Per raggiungere l'obiettivo userò principalmente `xarray`. In aggiunta userò `geopandas`, `datetime`, `pathlib` e `matplotlib`; quest'ultima per motivi di rappresentazione grafica dei dati.

In [ ]:
from pathlib import Path
from datetime import datetime
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

### Fonti dati

Per questa attività userò i dati prelavati dal CAMS - [Copernicus Atmosphere Monitoring Service](https://atmosphere.copernicus.eu/) e da ISTAT. Come vedremo, i dati ISTAT li ho usati per riuscire ad orientarmi meglio con le mappe prodotte.


In [ ]:
main_path = Path.cwd()
netcdf_path = main_path.joinpath('adaptor.cams_regional_fc.retrieve-1704441246.9534526-29460-11-5d3bb664-261d-4612-a637-a64b63cffaeb.nc')
boundaries_path = main_path.joinpath('Reg01012023_g').joinpath('Reg01012023_g_WGS84.shp')
pm10_img = main_path.joinpath('img_pm10')


## Contenuti
- [Il formato NetCDF](#il-formato-netcdf)
- [Workaround](#workaround)
- [PM10](#pm10)
- [Analisi delle 48h](#analisi-delle-48h)
- [Animazione dei dati](#animazione-dei-dati)
- [Conclusione](#conclusione)

## Il formato NetCDF

Il NetCDF è un formato file *array-oriented* spesso usato in campo scientifico, climatologico ed oceanografico. La particolarità di questo formato file sta nel fatto che i dati in esso contenuti, disposti sotto forma di matrici, riproducono sia la natura del fenomeno osservato che la sua temporalità. E' possibile storare in questo particolare tipo di formato file più fenomeni. Nel caso che andremo a vedere a breve, ad esempio, utilizzeremo un file `.nc` che contiene dati sia sul PM10 che sul SO2; entrambe le informazioni fanno riferimento allo stesso periodo temporale.

Per un approfondimento ulteriore ti rimando a [questa](https://confluence.ecmwf.int/display/CKB/What+are+NetCDF+files+and+how+can+I+read+them) pagina.

Di seguito utilizzerò `xarray` per leggere un NetCDF.

In [ ]:
netcdf_data = xr.open_dataset(netcdf_path)

netcdf_data

Il campo *Dimensions* contiene al suo interno il posizionamento geografico del dato, il riferimento temporale, che in questo caso corrisponde a 48h ed il `level` inteso come la quota di misurazione. Il dato che ho scaricato dal CAMS ha quindi una particolarità in più rispetto a quello che ho indicato in precedenza, oltre alla temporalità dello stesso c'è anche la quota altimetrica di misurazione(0, 50 e 100 metri). Come ho detto in precedenza, il formato NetCDF è *array-oriented*, può quindi gestire più domini purchè facciano riferimento alla stessa area geografica ed alla stessa temporalità.

Gli array li vediamo nel campo *Coordinates* e, in questo caso, vanno a definire il dominio spazio-temporale del dato.

Il dato in se, la concentrazione di inquinante, lo vediamo in *Data variables*.

*Indexes* ci mostra quali sono gli indici a cui possiamo fare riferimento per le nostre query.

*Attributes* è un campo abbastanza autoesplicativo.

## Workaround

Il periodo temporale di riferimento del dato che ho [scaricato](https://ads.atmosphere.copernicus.eu/cdsapp#!/search?type=dataset) è quello che va dal 31-12-2023 al 01-01-2024, le 48h riportate in *Dimensions*.

La variable temporale nel campo *Coordinates* è salvata come `timedelta64[ns]`, cioè un time delta in nanosecondi.

In [ ]:
netcdf_date_range = netcdf_data.coords['time']

netcdf_date_range

Ho dovuto quindi preventivamente convertirlo in un formato più comprensibile, il `datetime64`.

In [ ]:
netcdf_date_range = netcdf_date_range.values.astype('datetime64')

netcdf_date_range

Questo però ancora non è sufficiente ai miei scopi ed ho quindi creato una funzione che elimina i nanosecondi e mi restituisce la data e l'ora legata alla singola variabile. E' da notare che purtroppo questa tipologia di dato, nonostante il periodo temporale di riferimento fosse stato definito nel momento della ricerca dello stesso, parte dal 1 gennaio 1970.

In [ ]:
def from_datetime64_to_datetime(input_value) -> datetime:
    to_string = str(input_value)
    drop_nanoseconds = to_string.split('.')[0]

    output_value = datetime.strptime(drop_nanoseconds, "%Y-%m-%dT%H:%M:%S")

    return output_value


In [ ]:
min_date = from_datetime64_to_datetime(netcdf_date_range.min())
max_date = from_datetime64_to_datetime(netcdf_date_range.max())

min_date.date(), max_date.date()

Sono costretto quindi a crearmi una variabile `date_difference` che andrò ad usare per ottenere data ed ora di mio interesse grazie a `sensing_date`.

In [ ]:
sensing_date = datetime.strptime("2023-12-31", "%Y-%m-%d")

date_difference = sensing_date - min_date

date_difference

In [ ]:
date_range = []
for x in netcdf_date_range:
    to_datetime = from_datetime64_to_datetime(x)
    target_date = to_datetime + date_difference

    date_range.append(target_date)
    
    
date_range

Grazie a questo workaround posso finalmente vedere nel NetCDF il periodo temporale di mio interesse andando a sostituire quello calcolato con quello esistente nel file.

In [ ]:
new_netcdf_data = netcdf_data.assign_coords({'time': date_range})

new_netcdf_data

## PM10

Abbiamo in precedenza detto che il formato file in questione può contenere diverse variabili a cui possono essere collegate diverse quote e tempi.

Di seguito estrarrò i dati della sola PM10 per la prima ora di misurazione a quota 0 metri.

In [ ]:
single_data = new_netcdf_data[dict(level=[0], time=[0])]
single_data = single_data['pm10_conc']

single_data

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10, 10))

single_data.plot(
    ax=ax1,
    cmap='jet',
    robust=True
)

plt.show()

Per orientarmi meglio aggiungerò i confini regionali di [ISTAT al 2023](https://www.istat.it/it/archivio/222527) che essendo in 32632 devo riproiettare in 4326 per poterli vedere su mappa insieme con dato sulla PM10.

In [ ]:
boundaries_data = gpd.read_file(boundaries_path).to_crs('EPSG:4326')

boundaries_data

In [ ]:
fig2, ax2 = plt.subplots(figsize=(10, 10))
plt.title("Concentrazione di pm10")

single_data.plot(
    ax=ax2,
    cmap='jet',
)
boundaries_data.boundary.plot(
    ax=ax2,
    edgecolor='white',
    linewidth=0.75
)

plt.show()

L'immagine ci dice che all'1 di notte di sabato 31 dicembre 2023 in Pianura Padana si producevano più di 100 µg/m3 di PM10.

## Analisi delle 48h

In precedenza abbiamo capito come interagire con un NetCDF e coome estrarre i dati di nostro insteresse. Di seguito andrò a lavorare sul livello 0 metri e sull'intera temporalità del dato andando a generare una immagine per ora.

In [ ]:
netcdf_lvl0 = new_netcdf_data[dict(level=[0])]

pm10 = netcdf_lvl0['pm10_conc']

pm10

In [ ]:
min_pm10 = round(float(pm10.min().values), 2)
max_pm10 = round(float(pm10.max().values), 2)
mean_pm10 = round(float(pm10.mean().values), 2)
std_pm10 = round(float(pm10.std().values), 2)

f"Concentrazioni di PM10[µg/m3] min: {min_pm10}, max: {max_pm10}, mean: {mean_pm10}, std: {std_pm10}"

In [ ]:
for single_object in pm10:
    object_date = from_datetime64_to_datetime(single_object.coords['time'].values)
    map_title = f"Concentrazione di {single_object.attrs['species'].split(' ')[0]} a 0m di quota | {object_date}"
    map_subtitle = f"Tra il 31-12-2023 e 01-01-2024\n la concentrazione massima è stata {max_pm10} µg/m3\n e la minima {min_pm10} µg/m3"

    fig4, ax4 = plt.subplots(figsize=(10, 10))
    
    single_object.plot(
        ax=ax4,
        cmap='jet',
        vmin=min_pm10,
        vmax=max_pm10,
        #robust=True,
    )
    boundaries_data.boundary.plot(
        ax=ax4,
        edgecolor='white',
        linewidth=0.75
    )
    
    plt.title(map_subtitle)
    plt.suptitle(map_title)
    plt.savefig(pm10_img.joinpath(f"pm10_{object_date}.png"))
    plt.close()


## Animazione dei dati

In precedenza ho estratto e salvato in una apposita cartella le immagini orarie che riproducono la diffusione di PM10 sul territorio italiano nel periodo temporale di riferimento.

Mi è sembrato interessante fare uno step in più che esula dalla comprensione su come si gestisce un `.nc` e cioè provare ad animare le immagini creando una [GIF](https://it.wikipedia.org/wiki/Graphics_Interchange_Format). Per fare ciò mi servirà `Pillow` ed in particolare la funzione `save`; [qui](https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif) il link alla documentazione ufficiale.

In [ ]:
pm10_images = [img for img in pm10_img.glob(pattern='*.png')]

pm10_images

In [ ]:
print(f"Immagini totali: {len(pm10_images)}")

In [ ]:
from PIL import Image

def build_gif(imgs_folder, duration: int = 100):
    frame_list = [Image.open(img) for img in imgs_folder.glob(pattern='*.png')]

    duration_frame = [duration for frame in range(0, len(frame_list))]
    
    first_frame = frame_list[0]
    
    first_frame.save(
        fp=imgs_folder.joinpath('pm10.gif'), 
        save_all=True,
        append_images=frame_list[1:],
        duration=duration_frame,
        loop=None,
    )

In [ ]:
build_gif(
    imgs_folder=pm10_img,
    duration=500
    )

In [ ]:
from IPython.display import Image as img_open

img_open(open(pm10_img.joinpath('pm10.gif'), 'rb').read())

## Conclusione

Abbiamo visto non solo come poter leggere e gestire un file `.nc` ma anche come animare le immagini che eventualmente estraiamo dal file. Come abbiamo visto, la gestione di una file di questa tipologia non è immediata e spesso dobbiamo inventarci degli stratagemmi per poter ottenere in output quello che a noi interessa.